I WILL PERFORM ALL MY ONE HOT ENCODING OF CATEGORICAL FEATURES BEFORE CROSS VALIDATION PIPELINE, BUT ALL NUMERICAL PREPROCESSING WILL BE PERFORMED WITHIN.

In [99]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
df = pd.read_csv('/home/jovyan/data1030/data1030-oscars-prediction-project/data/movies.csv')

# parsing and one hot encoding of "weird" columns, unique due to it's structure
bar_cats =['New_York_Film_Critics_Circle_won_categories','Hollywood_Film_won_categories','Hollywood_Film_nominated_categories','Austin_Film_Critics_Association_won_categories','Austin_Film_Critics_Association_nominated_categories','Denver_Film_Critics_Society_won_categories','Denver_Film_Critics_Society_nominated_categories','Boston_Society_of_Film_Critics_won_categories','Boston_Society_of_Film_Critics_nominated_categories','New_York_Film_Critics_Circle_nominated_categories','Los_Angeles_Film_Critics_Association_won_categories','Los_Angeles_Film_Critics_Association_nominated_categories','Online_Film_Critics_Society_won_categories','Online_Film_Critics_Society_nominated_categories','People_Choice_won_categories','People_Choice_nominated_categories','London_Critics_Circle_Film_won_categories','London_Critics_Circle_Film_nominated_categories','American_Cinema_Editors_won_categories','American_Cinema_Editors_nominated_categories','Costume_Designers_Guild_won_categories','Costume_Designers_Guild_nominated_categories','Online_Film_Television_Association_won_categories','Online_Film_Television_Association_nominated_categories','Producers_Guild_won_categories','Producers_Guild_nominated_categories','Art_Directors_Guild_won_categories','Art_Directors_Guild_nominated_categories','Writers_Guild_won_categories','Writers_Guild_nominated_categories','Critics_Choice_won_categories','Critics_Choice_nominated_categories','Directors_Guild_won_categories','Directors_Guild_nominated_categories','Screen_Actors_Guild_won_categories','Screen_Actors_Guild_nominated_categories','BAFTA_won_categories','BAFTA_nominated_categories','Golden_Globes_won_categories','Golden_Globes_nominated_categories','Oscar_nominated_categories','genre']
bar_ohe = []
for label in bar_cats:
    df[label].fillna("nan", inplace=True)
    cat_series = df[label].apply(lambda x: (x.split('|')))
    cats = [cat  for ele in list(cat_series) for cat in ele]
    cat_set = set(cats)
    if 'nan' in cat_set:
        cat_set.remove('nan')
    total_cats = list(cat_set)
    cat_cols = [label + ": " + x for x in total_cats]

    mv_vals = np.asarray(cat_series)
    cat_ohe = np.zeros((len(cat_series),len(total_cats)))
    for i in range(len(mv_vals)):
        for j in range(len(total_cats)):
            cat_ohe[i,j] = total_cats[j] in mv_vals[i]
    ohe = pd.DataFrame(cat_ohe, columns = cat_cols)
    bar_ohe.append(ohe)
bar_ohe = pd.concat(bar_ohe,axis=1)  

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
oe = OrdinalEncoder(categories = ['No','Yes'])

#Replacing columns with "Yes"/"No" values to 1/0
ord_cols = 'Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated'
ord_cols = [x for x in ord_cols.split(',')]
for col in ord_cols:
    df[col].replace({'Yes':1,'No':0}, inplace=True)

ohe_data = pd.DataFrame(ohe.fit_transform(np.array(df['certificate'].replace({np.nan:"Missing"})).reshape(-1,1)))

ohe_data.columns= ohe.get_feature_names(['certificate'])

categorical_data = pd.concat([df[ord_cols],ohe_data,bar_ohe], axis=1)

In [100]:
le = LabelEncoder()
le_data = pd.DataFrame(le.fit_transform(df['Oscar_Best_Picture_won']))
le_data.columns=['Oscar_Best_Picture_won']
y = le_data['Oscar_Best_Picture_won']
y.value_counts()/len(y)

0    0.985425
1    0.014575
Name: Oscar_Best_Picture_won, dtype: float64

In [101]:
drop_cols =['Oscar_Best_Picture_won','certificate','movie_id','synopsis','New_York_Film_Critics_Circle_won_categories','Hollywood_Film_won_categories','Hollywood_Film_nominated_categories','Austin_Film_Critics_Association_won_categories','Austin_Film_Critics_Association_nominated_categories','Denver_Film_Critics_Society_won_categories','Denver_Film_Critics_Society_nominated_categories','Boston_Society_of_Film_Critics_won_categories','Boston_Society_of_Film_Critics_nominated_categories','New_York_Film_Critics_Circle_nominated_categories','Los_Angeles_Film_Critics_Association_won_categories','Los_Angeles_Film_Critics_Association_nominated_categories','Online_Film_Critics_Society_won_categories','Online_Film_Critics_Society_nominated_categories','People_Choice_won_categories','People_Choice_nominated_categories','London_Critics_Circle_Film_won_categories','London_Critics_Circle_Film_nominated_categories','American_Cinema_Editors_won_categories','American_Cinema_Editors_nominated_categories','Costume_Designers_Guild_won_categories','Costume_Designers_Guild_nominated_categories','Online_Film_Television_Association_won_categories','Online_Film_Television_Association_nominated_categories','Producers_Guild_won_categories','Producers_Guild_nominated_categories','Art_Directors_Guild_won_categories','Art_Directors_Guild_nominated_categories','Writers_Guild_won_categories','Writers_Guild_nominated_categories','Critics_Choice_won_categories','Critics_Choice_nominated_categories','Directors_Guild_won_categories','Directors_Guild_nominated_categories','Screen_Actors_Guild_won_categories','Screen_Actors_Guild_nominated_categories','BAFTA_won_categories','BAFTA_nominated_categories','Golden_Globes_won_categories','Golden_Globes_nominated_categories','Oscar_nominated_categories','genre','year','release_date','release_date.year', 'release_date.month', 'release_date.day-of-month', 'release_date.day-of-week']
drop_cols = drop_cols + ord_cols
df.drop(columns = drop_cols, inplace = True)
df_pp = pd.concat([df,categorical_data,y], axis=1)
df_pp.shape

(1235, 1019)

In [103]:
df_pp.to_csv('/home/jovyan/data1030/data1030-oscars-prediction-project/data/pre_training_data.csv', index=False)

In [107]:
df_pp['Oscar_Best_Picture_won'].value_counts()

0    1217
1      18
Name: Oscar_Best_Picture_won, dtype: int64